In [1]:
PPMC_ORDER = 4
ID = 0
TEXT = 9
PREDICTION = 6

fieldnames = ["Id", "ProductId", "UserId", "ProfileName", "HelpfulnessNumerator",
            "HelpfulnessDenominator", "Prediction", "Time", "Summary", "Text"]

import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import csv
from PPMC import PPMC

def compress_all(line):
    p = line[1][0]
    reviews = line[1][1]
    for review in reviews:
        p.compress(review)
    return (line[0], (line[1][0], len(reviews)))

#text = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."

trainRDD = sc.textFile("train_sin_repeticiones.csv", minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.reader(x))
trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

#trainRDD = trainRDD.sample(False, 0.01)

#se podria hacer un map antes que limpie el texto
#comprimo todo el train con ppmc
trainRDD = trainRDD.map(lambda line: (line[PREDICTION], (line[TEXT])))
trainRDD = trainRDD.reduceByKey(lambda value1, value2: (value1, value2))
trainRDD = trainRDD.map(lambda line: (line[0], (PPMC(PPMC_ORDER), line[1])))
trainRDD = trainRDD.map(lambda line: compress_all(line))


In [2]:
l=[0]

def foo(value1, value2):
    print l[0]
    l[0]+=1
    return (value1[0]+value2[0], value1[1]+value2[1])

test_prediction = open("test_prediction", "w")
writer = csv.DictWriter(test_prediction, fieldnames=fieldnames)
writer.writeheader()

with open("test.csv", "r") as test:
    reader = csv.DictReader(test)
    for row in reader:
        text = row["Text"]
        #comprimo el texto de prueba con los modelos de todo el rdd
        testRDD = trainRDD.map(lambda line: (line[0], (len(line[1][0].compress(text)), line[1][1])))
        testRDD = testRDD.reduce(lambda line1, line2: line1 if line1[1][0]/line1[1][1] < line2[1][0]/line2[1][1] else line2)
        row["Prediction"] = testRDD[0]
        writer.writerow(row)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 133, in dump_stream
    for obj in iterator:
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", line 1727, in add_shuffle_key
    yield outputSerializer.dumps(items)
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/

In [6]:
trainRDD = trainRDD.reduce(lambda line1, line2: line1 if line1[1][0]/line1[1][1] < line2[1][0]/line2[1][1] else line2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 11.0 failed 1 times, most recent failure: Lost task 1.0 in stage 11.0 (TID 16, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", line 792, in func
    initial = next(iterator)
TypeError: <lambda>() takes exactly 2 arguments (1 

In [5]:
print trainRDD.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 13, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/eugenia/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", line 1293, in takeUpToNumLeft
    yield next(iterator)
TypeError: <lambda>() takes exactly 2 arguments (1 giv